**Note: The following comes from [Use Word Embedding Layers - Deep Learning Keras](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/).**
# Training Your Own Embeddings

You'll need to make sure that `tensorflow` and `keras` are installed:
```
pip install tensorflow keras
```

### Note:
The following is NOT implementing `word2vec`. It is simply training an embedding layer via a vanilla neural network.

In [1]:
from numpy import array


In [2]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Using TensorFlow backend.
/Users/akshaybhide/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akshaybhide/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akshaybhide/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/akshaybhide/opt/ana

In [ ]:
##have docs, have the sentiments for each of tem
##want to predict sentiment based on these docs
# define documents
docs = ['Well done!',
        'Good work',
        "Awesome job",
        "Amazing",
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good really bad',
        'poor work',
        "Weak, not well done",
        'Poor job',
        'Weak and terrible',
        'Not very good',
        'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0]) # 1 means it is positive, 0 means it is negative

## Define the Vocab Size

In [ ]:
# you set the vocabulary size to some number that represents the total number of unique words in your vocabulary
vocab_size = 50
##less than 50 unique words in docs, but want to have space for some special tokens Keras will make

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
##tokenizing
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
tokenizer.word_index
##output shows index positions for each word
##0 is empty string/token
##index 2 has embedding of that word

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

## Integer Encode the Documents

In [ ]:
##for each document, you want it to be a list of integers
from typing import List
def integer_encode_documents(docs: List[str], tokenizer: Tokenizer)-> List[List[int]]:
    documents = []
    for d in docs:
        doc_integers = []
        for i in text_to_word_sequence(d):
            doc_integers.append(tokenizer.word_index[i])
        documents.append(doc_integers)
    return documents

In [ ]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [ ]:
# integer encode the documents
encoded_docs = integer_encode_documents(docs, tokenizer)
# this is a list of lists, the numbers represent the index position of that word.
# for instance, 33 means the 33rd word in the vocabulary
# Notice the last document has 4 numbers, since it is a 4 word document: Could have done better.
from pprint import pprint
pprint(encoded_docs)
##very last doc in prof example is 20 21 1 22 (could have done better), 4 tokens, so we get 4 numbers

## Get Max Length of Documents

In [ ]:
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

In [ ]:
# get the max length in terms of token length
max_length = get_max_token_length_per_doc(docs)

## Pad Documents to Max Length

In [ ]:
##eventually want to make a matrix, so need consistent shape for each doc

# pad documents to a max length of 4 words
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_docs)
# since the max length is 4 words in a document, we pad all the documents to have 4 words, just set index to 0
# if it doesn't have any words

Note: important design consideration - pad zeros after the document, or before?

In [ ]:
##after, look up embeddings for each index in each list
##shape is s x d, sequence length s is 4, d is # of embedding dimensions

##look at Colab notebook for the image that was here, for getting a document's embeddings

## Define an Embedding Size
This represents how many numbers will "represent" a word. In `word2vec`, this would be 100, or 300.

In [ ]:
EMBEDDING_SIZE = 8

# Define Our Deep Learning Model

In [ ]:
# define the model
# remember, vocab_size = 50
##use sequential API in Keras
model = Sequential()
##add layers to model - embedding layer (size 50 by embedding size of 8, third dimension is sequence length 4)
##it's 50 x 8 x 4
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
model.add(Flatten()) 
# for each document, the output of the embedding layer is 4 x 8 matrix
# (4 since 4 words per document, 8 since size 8 embedding). Flatten makes this a 32 x 1 vector.
##use sigmoid activation for the very end of the dense layer to get something between 0 and 1
model.add(Dense(1, activation='sigmoid')) 
# these 32 elements are coalesced into one final output node, a sigmoid
# that outputs a probability of positive or negative

!["ada"](https://raw.githubusercontent.com/ychennay/dso-560-nlp-text-analytics/main/images/architecture.png "Logo Title Text 1")

In [ ]:
##need N scalar values in the end, final layer should be N x 1, it's outputting a bunch of scores
##goes from 32 numbers for each document to 1 for each document

In [ ]:
# compile the model
##binarycrossentropy is large if you're really wrong, if you're close then the loss will be really small
##use that to update weights over and over again
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()
## #params/total params is number of weights to update

In [ ]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

### Get Embedding Layer Weights

In [ ]:
embedding_layer = model.layers[0]
embedding_layer.get_weights()[0].shape
## should be 50x8, vocab size x embedding dimension
##check recording a little before 7:14
##doing embedding_layer[,:2] or something should give embeddings for index 2 word, check recording ^^^

## Test Prediction

In [ ]:
encoded_test_docs = integer_encode_documents(["Awesome work", 
                                              "Really bad, terrible", 
                                              "amazing work"], tokenizer)

# pad test documents
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_test_docs)

In [3]:
prediction = model.predict(padded_test_docs, verbose=0)
prediction
##predictions for first and third are more than 0.5, for second its lower then 0.5
##something is getting learned, probably overfitting "amazing" and "awesome", but is giving us some useful information

NameError: name 'model' is not defined

In [ ]:
##this is showing how you can create your own embeddings if you want to
##you also have pre-trained embeddings available to you
##in real life if you are in a custom domain, the corpus that word2vec is trained on maybe be too general for your 
##use case

##you could create your own embeddings if you wanted to in that case

# Using Pre-Trained Embeddings

In [ ]:
##check colab notebook, might be some different stuff here

In [ ]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print("Encoded docs:\n", encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Padded docs:\n", padded_docs)
# load the whole embedding into memory
embeddings_index = dict()

In [ ]:
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    ##word is first word, everything else is the embeddings themselves
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
##colab notebook has looking up "happy" embedding

## Pre-Load the Weight Matrix

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [ ]:
# define model
model = Sequential()
##trainable = False means we don't want model to update Glove embeddings, they should be fixed/have already been
##trained
##pretty much everything else is the same
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

!["ada"](https://raw.githubusercontent.com/ychennay/dso-560-nlp-text-analytics/main/images/architecture2.png "Logo Title Text 1")

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))
##100% accuracy but we only have 10 documents

In [ ]:
model.predict(padded_docs)
##first 5 were positive last 5 were negative, does a pretty good job with it
##used with Glove embeddings

In [ ]:
encoded_docs

In [ ]:
from typing import List

In [ ]:
encoded_test_docs = integer_encode_documents(["Good work"])

# pad test documents
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_test_docs)
model.predict(padded_test_docs)

## Word Embeddings with Amazon Toy Reviews Dataset

In [1]:
import numpy as np

NUM_SAMPLES = 5000

good_reviews = open("../week1/good_amazon_toy_reviews.txt").readlines()
bad_reviews = open("../week1/poor_amazon_toy_reviews.txt").readlines()

sampled_good_reviews = good_reviews[:NUM_SAMPLES]
sampled_bad_reviews = bad_reviews[:NUM_SAMPLES]

docs = sampled_good_reviews + sampled_bad_reviews
labels = np.concatenate([np.ones(NUM_SAMPLES), np.zeros(NUM_SAMPLES)])

## Remove Stopwords Using Spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
stopwords_removed_docs = list(
    map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), docs))

## Tokenize the Text
I'm just using perhaps the most basic tokenization possible from Keras. Read [the documentation for more options](https://keras.io/preprocessing/text/). The most notable options:
* `num_words`: the maximum number of words to keep, based on word frequency.
* `oov_token`: adds a `OOB` (out of bag) or `OOV` (out of vocabulary) token to the `word_index`.

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(stopwords_removed_docs)

In [ ]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [ ]:
# integer encode the documents
encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
# this is a list of lists, the numbers represent the index position of that word.
# for instance, 33 means the 33rd word in the vocabulary
# Notice the last document has 4 numbers, since it is a 4 word document: Could have done better.
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
vocab_size = int(len(tokenizer.word_index) * 1.3)
print(f"Vocab size is {vocab_size} unique tokens.")

In [ ]:
EMBEDDING_SIZE = 50

In [ ]:
from keras.utils.vis_utils import plot_model

# Define and Compile the Model

In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
model.add(Flatten()) 
# for each document, the output of the embedding layer is 4 x 8 matrix
# (4 since 4 words per document, 8 since size 8 embedding). Flatten makes this a 32 x 1 vector.
model.add(Dense(1, activation='sigmoid')) 
# these 32 elements are coalesced into one final output node, a sigmoid
# that outputs a probability of positive or negative
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)


## Fit the Model

In [ ]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
final_embeddings = model.layers[0].get_weights()[0]

In [ ]:
final_embeddings.shape

In [ ]:
final_embeddings = final_embeddings[:len(tokenizer.word_index)]

In [ ]:
embeddings_dict = {token: embedding for token, embedding in zip(tokenizer.word_index, final_embeddings)}

# Find Similarities of the new Embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarities = pd.DataFrame(cosine_similarity(final_embeddings), 
                            index=tokenizer.word_index, columns=tokenizer.word_index)

In [ ]:
# unstack matrix into table
similarity_table = similarities.rename_axis(None).rename_axis(None, axis=1).stack().reset_index()
# rename columns
similarity_table.columns = ["word1", "word2", "similarity"]
similarity_table.shape

similarity_table = similarity_table[similarity_table["similarity"] < 0.99]
similarity_table.shape

In [ ]:
similarity_table.sort_values(by="similarity", ascending=False).drop_duplicates(
    subset="similarity", keep="first").head(30)